# RAG(Retrieval-Augmented Generation)

검색 증강 생성(RAG)은 근거 데이터를 제공하는 정보 검색 시스템을 추가하여 ChatGPT와 같은 대규모 언어 모델(LLM)의 기능을 보강하는 아키텍처입니다.

정보 검색 시스템을 추가하면 LLM 응답시 사용하는 근거 데이터를 제어할 수 있습니다.

엔터프라이즈 솔루션의 경우, RAG 아키텍처는 벡터화된 문서와 이미지, 그리고 해당 컨텐츠에 대한 임베딩(embedding) 모델이 있는 경우 기타 데이터 소스로부터 수집 된 엔터프라이즈 컨텐츠로 생성형 AI를 제한할 수 있다는 것을 의미합니다.

더 자세한 내용은 [Azure AI 검색의 RAG](https://learn.microsoft.com/ko-kr/azure/search/retrieval-augmented-generation-overview), [Azure AI 검색을 사용하여 RAG 솔루션을 빌드하는 방법](https://learn.microsoft.com/ko-kr/azure/search/tutorial-rag-build-solution) 에서 확인 할 수 있습니다.

![RAG architecture diagram](https://learn.microsoft.com/ko-kr/azure/search/media/retrieval-augmented-generation-overview/architecture-diagram.png#lightbox)

### RAG

이 예제에서는 이전 [vector_search](./01.vector_search.ipynb)에서 생성 한 `hotel-quickstart-vector' 인덱스를 이용합니다.

In [2]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

load_dotenv(override=True)

endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) if len(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) > 0 else DefaultAzureCredential()

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY", "") if len(os.getenv("AZURE_OPENAI_KEY", "")) > 0 else None
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002")
azure_openai_embedding_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 1536))
azure_openai_chat_deployment = os.getenv("AZURE_OPENAI_CRITIC_DEPLOYMENT", "gpt-4o")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01")

blob_container_name = os.getenv("AZURE_BLOB_CONTAINER_NAME", "aisearch-sample")
blob_connection_string = os.getenv("AZURE_BLOB_CONNECTION_STRING")

index_name = "hotel_quickstart_vector"

### Chat 모델을 이용하여 검색.

채팅을 위해 쿼리, 쿼리 응답 및 프롬프트를 LLM에 보냅니다. Gen AI를 사용하여 응답을 생성하며, 작업을 위해 클라이언트 설정, 검색 쿼리 설정, LLM 호출 및 응답 가져오기 등 세가지 작업으로 나누었습니다.

더 자세한 내용은 [자습서: 채팅 모델을 사용하여 데이터 검색(Azure AI 검색의 RAG)](https://learn.microsoft.com/ko-kr/azure/search/tutorial-rag-build-solution-query)을 참조하세요.

In [3]:
openai_client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

search_client = SearchClient(
    endpoint=endpoint, 
    index_name=index_name, 
    credential=credential
)


# prompt
GROUNDED_PROMPT="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
Use bullets if the answer has multiple points.
If the answer is longer than 3 sentences, provide a summary.
Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Answer as Korean if you ask a question in Korean.
Query: {query}
Sources:\n{sources}
"""

In [7]:
# 검색 쿼리. 
# 10개의 검색 결과를 찾는 쿼리를 생성합니다.
query="Times Square 근처의 호텔을 두세곳 추천해 주세요."
vector_query = VectorizableTextQuery(
    text=query, 
    k_nearest_neighbors=50,
    fields="descriptionVector")

# 검색 결과는 상위 5개 항목으로 제한됩니다.
# 결과의 수를 제한하면 LLM 쿼터를 유지하는 데 도움이 될 수 있습니다.
search_results = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["HotelName", "Description", "Address"],
    top=5,
)

# 소스를 구분하기 위해 고유한 구분 기호를 사용합니다. 
# 소스 문서에 이 순서가 포함될 가능성이 낮기 때문에 등호(=) 뒤에 개행이 반복되도록 하였습니다.
sources_formatted = "=================\n".join([f'HotelName: {document["HotelName"]}, Description: {document["Description"]}, Address: {document["Address"]}' for document in search_results])

In [8]:
response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=azure_openai_chat_deployment
)

print(response.choices[0].message.content)

- Stay-Kay City Hotel: 이 호텔은 뉴욕의 주요 상업 거리에 위치한 클래식 호텔로, 완전히 개보수 되었으며 타임스퀘어 및 뉴욕의 역사적인 중심지에서 몇 분 거리에 있습니다. (주소: 677 5th Ave, New York, NY 10022, USA)

- By the Market Hotel: 이 호텔은 첼시 지역에 위치하고 있으며 타임스퀘어 및 엠파이어 스테이트 빌딩에서 도보 거리에 있습니다. 새로 지어진 객실과 뛰어난 서비스를 제공합니다. (주소: 11 Times Sq, New York, NY 10036, USA)


### 다른 예제 - 1

검색 엔진과 LLM이 더 세분화된 정보를 찾아야 하는 또 다른 질문을 해 보겠습니다.

In [11]:
# 수영장과 다운타운이 가까운 호텔로 쿼리를 변경합니다.

query="수영장이 있는 호텔은 어디인가요? 다운타운과 가까운 곳이면 좋겠습니다."
vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=50,
    fields="descriptionVector")

search_results = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["HotelName", "Description", "Address"],
    top=5,
)

sources_formatted = "=================\n".join([f'HotelName: {document["HotelName"]}, Description: {document["Description"]}, Address: {document["Address"]}' for document in search_results])

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=azure_openai_chat_deployment
)

print(response.choices[0].message.content)

다운타운과 가까우며 수영장이 있는 호텔로는 "Hotel on the Harbor"와 "King's Cellar Hotel"이 있습니다.

- "Hotel on the Harbor": 실내 수영장과 사우나를 보유하고 있으며 극장과 박물관, 컨벤션 센터에 가까이 위치해 있습니다. ([출처](hotelname: Hotel on the Harbor))
- "King's Cellar Hotel": 실내 옥상 수영장이 있으며, 유명한 다운타운 명소에서 몇 걸음 떨어져 있습니다. ([출처](hotelname: King's Cellar Hotel))


### 다른 예제 - 2

In [25]:
query="Spa 가 제공되는 호텔 곳을 추천해 주세요. 호텔 설명, 주소, 위치, 4인 1실의 객실 요금을 알려주세요."
# query="Can you recommend a few hotels that offer complimentary breakfast? Tell me their description, address, tags, and the rate for one room they have which sleep 4 people."

# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
selected_fields = ["HotelName", "Description", "Address", "Rooms", "Tags"]
search_results = search_client.search(
    search_text=query,
    top=5,
    select=selected_fields,
    query_type="semantic",
    semantic_configuration_name="my-semantic-config"
)
sources_filtered = [{field: result[field] for field in selected_fields} for result in search_results]
sources_formatted = "=================\n".join([f'HotelName: {document["HotelName"]}, Description: {document["Description"]}, Address: {document["Address"]}, Rooms: {document["Rooms"]}, Tags: {document["Tags"]}' for document in sources_filtered])

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=azure_openai_chat_deployment
)

print(response.choices[0].message.content)

- **호텔 이름**: Gold View Inn  
  - **설명**: AAA Four Diamond Resort. 아름답게 조경된 6에이커 위에 자리잡고 있으며, 공원에서 2블록 떨어진 곳에 위치하고 있습니다. 스파에서 휴식을 취하고, 현장에서 아트 투어를 즐길 수 있습니다.
  - **주소**: 1414 NW Northrup St, Portland, OR 97209, USA
  - **4인 1실 요금**: 호텔에서 제공하는 4인 1실 객실 정보는 제공되지 않았습니다.

- **호텔 이름**: White Mountain Lodge & Suites  
  - **설명**: 숲의 중심에서 자연과 더불어 살 수 있는 장소입니다. 넓은 트레일 시스템을 따라 하이킹을 하고, 자연 온천을 방문하거나 일요일 저녁에 특별 게스트 뮤지션이나 시 낭독을 즐길 수 있습니다.
  - **주소**: 3000 E 1st Ave, Denver, CO 80206, USA
  - **4인 1실 요금**: Standard Room, 2 Queen Beds (Cityside), BaseRate: $136.99 per night

- **호텔 이름**: Ocean Water Resort & Spa  
  - **설명**: 인생의 휴가를 위한 새로운 럭셔리 호텔입니다. 모든 방에서는 만의 전망을 볼 수 있으며, 부두 근처에 위치하고 있고, 옥상 수영장과 수변 식사를 제공합니다.
  - **주소**: 5426 Bay Center Dr, Tampa, FL 33609, USA
  - **4인 1실 요금**: Deluxe Room, 2 Queen Beds (Cityside), BaseRate: $151.99 per night

이 호텔들은 각각 스파 서비스를 제공하며, 각기 다른 특색과 위치에 따라 다양한 경험을 선사합니다. 객실 정보는 호텔마다 상이할 수 있습니다.
